시계열 데이터 분석 방법

1. 분해법
시계열 데이터가 가지는 추세, 계절성, 주기 및 불규칙성과 같은 구성요소를 분해하여 분석한다.
2. 시간 영역 분석법(ARIMA)
시간 영역 분석을 기반으로 하는 확률적 시계열 분석 방법
현재의 데이터 값이 이전의 데이터 값과 일정한 관계를 가지고 있다는 가정 하에 모델링을 수행한다.

# 1.3 시계열 데이터 전처리 방법

In [1]:
# datetime 변환
import pandas as pd
d=pd.DataFrame({'date':['2019-01-03', '2021-11-22', '2023-01-05'], #날짜가 object형이면 datetime형으로 변경한다.
'name':['J', 'Y', 'O']})
print(d.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   name    3 non-null      object
dtypes: object(2)
memory usage: 180.0+ bytes
None


In [4]:
d['date'] = pd.to_datetime(d.date, format= '%Y-%m-%d')
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3 non-null      datetime64[ns]
 1   name    3 non-null      object        
dtypes: datetime64[ns](1), object(1)
memory usage: 180.0+ bytes


In [5]:
#datetime 형의 컬럼을 인덱스로 설정
#시계열 데이터는 시간에 종속되기 때문에 시간 컬럼이 인덱스로 설정되는 것이 좋다.

d.set_index(keys=['date'], inplace=True)
print(d)

           name
date           
2019-01-03    J
2021-11-22    Y
2023-01-05    O


In [11]:
#결측치 처리
import numpy as np
d=pd.DataFrame({'date':['2019-01-03', '2021-11-22','2021-12-01','2023-01-05'],
'x1':[0.1,2.0,np.nan,1.2]})

d['date']=pd.to_datetime(d.date,format='%Y-%m-%d') #datetime형으로 변환

d.set_index(keys=['date'],inplace=True) #date를 인덱스로

print(d.isnull().sum()) #결측치 확인
print(d)

x1    1
dtype: int64
             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2021-12-01  NaN
2023-01-05  1.2


In [14]:
# 1. 이전 값으로 결측치 채우기
d = d.fillna(method='ffill') #forward fill
print(d)

             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2021-12-01  2.0
2023-01-05  1.2


In [16]:
# 2. 결측치 제거
d=pd.DataFrame({'date':['2019-01-03', '2021-11-22','2021-12-01','2023-01-05'],
'x1':[0.1,2.0,np.nan,1.2]})
d['date']=pd.to_datetime(d.date,format='%Y-%m-%d')
d.set_index(keys=['date'],inplace=True)
print(d)

d = d.dropna()
print(d)

             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2021-12-01  NaN
2023-01-05  1.2
             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2023-01-05  1.2


In [17]:
# 3. 전후 값의 평균치로 채우기
d=pd.DataFrame({'date':['2019-01-03', '2021-11-22','2021-12-01','2023-01-05'],
'x1':[0.1,2.0,np.nan,1.2]})
d['date']=pd.to_datetime(d.date,format='%Y-%m-%d')
d.set_index(keys=['date'],inplace=True)
print(d)

d = d.interpolate()
print(d)

             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2021-12-01  NaN
2023-01-05  1.2
             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2021-12-01  1.6
2023-01-05  1.2


In [20]:
# 빈도 설정
# 빈도: 데이터 분석에서 설정하는 시간의 최소 단위
d=pd.DataFrame({'date':['2019-01-03', '2021-11-22','2021-12-01','2023-01-05'],
'x1':[0.1,2.0,1.6,1.2]})
d['date']=pd.to_datetime(d.date,format='%Y-%m-%d')
d.set_index(keys=['date'],inplace=True)

#인덱스 속성확인
print(d.index)
#빈도(주기)에 대해 설정한 것이 없어 freq가 None으로 설정되어 있다.

DatetimeIndex(['2019-01-03', '2021-11-22', '2021-12-01', '2023-01-05'], dtype='datetime64[ns]', name='date', freq=None)


In [24]:
print(d)

d2 = d.asfreq('Y', method='ffill') # 'Y' 매년 마지막 일
print(d2)

             x1
date           
2019-01-03  0.1
2021-11-22  2.0
2021-12-01  1.6
2023-01-05  1.2
             x1
date           
2019-12-31  0.1
2020-12-31  0.1
2021-12-31  1.6
2022-12-31  1.6


In [25]:
# 특징량 만들기
# rolling()을 사용하여 데이터를 shift하면서 부분 시계열을 추출해 그 부분의 통계량을 특징량으로 한다.
import numpy as np
d=pd.DataFrame({'date':['2021-01-06', '2021-01-13', '2021-01-20', '2021-01-27', '2021-02-03'],
'x1':[5,4,3,2,7]})
d['date']=pd.to_datetime(d.date)
d.set_index(keys=['date'],inplace=True)

print("원본")
print('=='*20)
print(d)
print('=='*20)
tmp=d.rolling(1).mean()
print(tmp)
print()
print('=='*20)
tmp=d.rolling(2).mean()
print(tmp)

원본
            x1
date          
2021-01-06   5
2021-01-13   4
2021-01-20   3
2021-01-27   2
2021-02-03   7
             x1
date           
2021-01-06  5.0
2021-01-13  4.0
2021-01-20  3.0
2021-01-27  2.0
2021-02-03  7.0

             x1
date           
2021-01-06  NaN
2021-01-13  4.5
2021-01-20  3.5
2021-01-27  2.5
2021-02-03  4.5


In [26]:
#이전 값과 차이 계산
#동일한 컬럼 값으로 채우기
import numpy as np
d=pd.DataFrame({'date':['2021-01-06', '2021-01-13', '2021-01-20', '2021-01-27', '2021-02-03'], 'x1':[5,4,3,2,7]})
d['date']=pd.to_datetime(d.date)
d.set_index(keys=['date'],inplace=True)

print("원본")
print('=='*20)
print(d)

y_diff=d.diff() # 이전값-이후값
y_diff.columns=['diff'] # 컬럼명 만들기
temp = pd.concat([d,y_diff],axis=1) # 데이터프레임 열끼리 합치기
print('=='*20)
print(temp)

원본
            x1
date          
2021-01-06   5
2021-01-13   4
2021-01-20   3
2021-01-27   2
2021-02-03   7
            x1  diff
date                
2021-01-06   5   NaN
2021-01-13   4  -1.0
2021-01-20   3  -1.0
2021-01-27   2  -1.0
2021-02-03   7   5.0


In [28]:
# 지연값 추출

import numpy as np
d=pd.DataFrame({'date':['2021-01-06', '2021-01-13', '2021-01-20', '2021-01-27', '2021-02-03'],
'x1':[5,4,3,2,7]})
d['date']=pd.to_datetime(d.date)
d.set_index(keys=['date'],inplace=True)

print("원본")
print('=='*20)
print(d)
print('=='*20)
d['shift']=d['x1'].shift(2) #데이터가 2개씩 뒤로 밀림
print(d)
print('=='*20)
d=d.fillna(method='bfill') # backward fill
print(d)

원본
            x1
date          
2021-01-06   5
2021-01-13   4
2021-01-20   3
2021-01-27   2
2021-02-03   7
            x1  shift
date                 
2021-01-06   5    NaN
2021-01-13   4    NaN
2021-01-20   3    5.0
2021-01-27   2    4.0
2021-02-03   7    3.0
            x1  shift
date                 
2021-01-06   5    5.0
2021-01-13   4    5.0
2021-01-20   3    5.0
2021-01-27   2    4.0
2021-02-03   7    3.0


In [29]:
#원-핫 인코딩 : 범주형 변수를 컴퓨터가 처리할 수 있도록 변환
#판다스 : get_dummies()

import numpy as np
d=pd.DataFrame({'date':['2021-01-06', '2021-01-13', '2021-01-20', '2021-01-27', '2021-02-03'], 'x1':[5,4,3,2,7],'과목':['a','b','c','d','e']})
d['date']=pd.to_datetime(d.date)
d.set_index(keys=['date'],inplace=True)
print("원본")
print('=='*20)
print(d)
x=pd.get_dummies(d['과목'])
x=pd.concat([d,x],axis=1)
print('=='*20)
print(x)

원본
            x1 과목
date             
2021-01-06   5  a
2021-01-13   4  b
2021-01-20   3  c
2021-01-27   2  d
2021-02-03   7  e
            x1 과목  a  b  c  d  e
date                            
2021-01-06   5  a  1  0  0  0  0
2021-01-13   4  b  0  1  0  0  0
2021-01-20   3  c  0  0  1  0  0
2021-01-27   2  d  0  0  0  1  0
2021-02-03   7  e  0  0  0  0  1
